<a href="https://colab.research.google.com/github/BogdanJas/vlm_learning/blob/main/vlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers opencv-python-headless

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import LlavaNextVideoProcessor, LlavaNextVideoForConditionalGeneration, BitsAndBytesConfig
import torch.quantization
import cv2
import numpy as np

In [ ]:
model_id = "llava-hf/LLaVA-NeXT-Video-7B-hf"
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=8.0
    ),
    device_map="auto",
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
processor = LlavaNextVideoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

In [ ]:
def read_video_opencv(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    indices = np.linspace(0, total_frames-1, num_frames, dtype=int)
    frames = []
    timestamps = []

    for frame_idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
            timestamps.append(frame_idx / fps)

    cap.release()
    return np.array(frames), timestamps

In [ ]:
def query_video(question, video_path):
    # Read video frames
    frames, timestamps = read_video_opencv(video_path)

    if len(frames) == 0:
        return "Error: Could not read video frames", []

    conversation = [{
        "role": "user",
        "content": [
            {"type": "text", "text": question},
            {"type": "video"},
        ],
    }]

    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs_video = processor(
        text=prompt,
        videos=frames,
        padding=True,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(**inputs_video, max_new_tokens=100, do_sample=False)
    response = processor.decode(output[0][2:], skip_special_tokens=True)

    return response


In [ ]:
video_path = '/content/video.mp4'
question = "What is the name of manufacture from the video?"
response, time_stamps = query_video(question, video_path)
print(f"Response: {response}")
print(f"Timestamps: {time_stamps}")

Expanding inputs for image/video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Expanding inputs for image.video tokens in LLaVa-NeXT-Video should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


Response: ER: 
What is the name of manufacture from the video? ASSISTANT: The logo
Timestamps: ['0.00s']
